In [29]:
import json
from easydict import EasyDict as edict
import pprint
import datetime
from py2neo import Graph, Node, NodeMatcher, Relationship
import pandas as pd
import numpy as np

In [2]:
album = "lotr"
title_of_image = "Frodo"
description = "Hobbit"
url = "images/werner/lotr/74bffb8ab937853bce9eabd2033456f4bf478ad2-b17c-434b-bb76-1a82aa4a2f46.jpg"
thumb_url = "images/werner/lotr/74bffb8ab937853bce9eabd2033456f442022bb6-b8e3-40e5-82c4-161557fdc73c_thumbnail_.jpg"

In [3]:
with open('nodes.json', 'r') as myfile:
    data=myfile.read()
nodes_json_loaded = json.loads(data)
myfile.close()

FileNotFoundError: [Errno 2] No such file or directory: 'nodes.json'

In [9]:
NEO4J_BOLT ='bolt://localhost:11005'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'skripsie'
GRAPH_INIT = Graph(NEO4J_BOLT, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))


In [31]:
get_album_image_properties("test")

,description,thumb_url,title,url
0,test,images/werner/test/1_test__thumbnail_Screensho...,Screenshotfrom2019-08-0318-34-3628122648-8af9-...,images/werner/test/1_test_Screenshot_from_2019...
1,test,images/werner/test/2_test__thumbnail_Screensho...,Screenshotfrom2019-08-0318-34-33224a83b6-109b-...,images/werner/test/2_test_Screenshot_from_2019...
2,test,images/werner/test/3_test__thumbnail_wood66f1b...,wood0b3a6328-d0ef-4b21-b72c-cf7692fc6e02.png,images/werner/test/3_test_wood0b3a6328-d0ef-4b...


In [65]:
def generate_gallery_json(album):
    """
    Generates gallery.json that the dashboard requires 
    to display images.
    Returns json of album image.
    """
    gallery_file_json_output_album={}
    gallery_file_json_output_album["name"] = album
    gallery_file_json_output_photos = generate_list_images(album)
    merged ={}
    merged["album"] = gallery_file_json_output_album
    merged["photos"] = gallery_file_json_output_photos

    with open("gallery.json","w") as f:
        json.dump(merged,f,indent=4)


    return json.dumps(merged)
    



In [66]:
def generate_list_images(album):
    """
    Generates gallery.json that the dashboard requires 
    to display images.
    """
    i=0
    listofImages = []
    df = get_album_image_properties(album)
    for index, row in df.iterrows():
        i+= 1
        dict_image = generate_images_dictionary(i,row['title'],row['description'],row['url'],row['thumb_url'])
        listofImages.append(dict_image)
    
    return listofImages

In [67]:
def generate_images_dictionary(ID_in,title_of_image,description,url,thumb_url):
    """
    Generates a dictionary based on image properties.
    """
    images_dictionary = {}
    now = datetime.datetime.now()
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    
    images_dictionary['id'] = ID_in
    images_dictionary['url'] = url
    images_dictionary['thumb_url'] = thumb_url
    images_dictionary['title'] = title_of_image
    images_dictionary['date'] = date_time
    images_dictionary['description'] = description
    
    return images_dictionary

In [68]:
def get_album_image_properties(album):
    """
    Returns:
    All image properties need to generate display json
    based on album input.
    """
    create_get_image_cypher = (("MATCH (a:Image)"
                            " WHERE a.album ='%s' "
                            "RETURN a.image_title as title,a.image_location as url,"
                            "a.image_thumbnail_location as thumb_url,"
                            "a.description as description")
                           % (album))

    
    df = GRAPH_INIT.run(create_get_image_cypher).to_data_frame()
    return df

In [69]:
generate_gallery_json('test')

'{"album": {"name": "test"}, "photos": [{"id": 1, "url": "images/werner/test/1_test_Screenshot_from_2019-08-03_18-34-3628122648-8af9-4768-afcc-980fbdc6ee05.png", "thumb_url": "images/werner/test/1_test__thumbnail_Screenshot_from_2019-08-03_18-34-36911d7f4f-93be-4e92-81f0-fcf4919fcecb.png", "title": "Screenshotfrom2019-08-0318-34-3628122648-8af9-4768-afcc-980fbdc6ee05.png", "date": "08/06/2019, 20:11:54", "description": "test"}, {"id": 2, "url": "images/werner/test/2_test_Screenshot_from_2019-08-03_18-34-33224a83b6-109b-4ccb-a464-38e850fe6cda.png", "thumb_url": "images/werner/test/2_test__thumbnail_Screenshot_from_2019-08-03_18-34-335967dc07-8de0-4c47-93e8-1b68eabc50b4.png", "title": "Screenshotfrom2019-08-0318-34-33224a83b6-109b-4ccb-a464-38e850fe6cda.png", "date": "08/06/2019, 20:11:54", "description": "test"}, {"id": 3, "url": "images/werner/test/3_test_wood0b3a6328-d0ef-4b21-b72c-cf7692fc6e02.png", "thumb_url": "images/werner/test/3_test__thumbnail_wood66f1be08-761f-4dc3-95ae-b1ef5d

In [5]:

print(generate_gallery_json(album,title_of_image,description,url,thumb_url,"gallery.json"))

[{'id': '1', 'url': 'images/werner/lotr/74bffb8ab937853bce9eabd2033456f4bf478ad2-b17c-434b-bb76-1a82aa4a2f46.jpg', 'thumb_url': 'images/werner/lotr/74bffb8ab937853bce9eabd2033456f442022bb6-b8e3-40e5-82c4-161557fdc73c_thumbnail_.jpg', 'title': 'Frodo', 'date': '08/06/2019, 19:33:38', 'description': 'Hobbit'}, {'id': '1', 'url': 'images/werner/lotr/74bffb8ab937853bce9eabd2033456f4bf478ad2-b17c-434b-bb76-1a82aa4a2f46.jpg', 'thumb_url': 'images/werner/lotr/74bffb8ab937853bce9eabd2033456f442022bb6-b8e3-40e5-82c4-161557fdc73c_thumbnail_.jpg', 'title': 'Frodo', 'date': '08/06/2019, 19:33:38', 'description': 'Hobbit'}]
{
    "album": {
        "name": "lotr"
    },
    "photos": [
        {
            "id": "1",
            "url": "images/werner/lotr/74bffb8ab937853bce9eabd2033456f4bf478ad2-b17c-434b-bb76-1a82aa4a2f46.jpg",
            "thumb_url": "images/werner/lotr/74bffb8ab937853bce9eabd2033456f442022bb6-b8e3-40e5-82c4-161557fdc73c_thumbnail_.jpg",
            "title": "Frodo",
          

In [55]:
"""
Generates node_json_output.json that the dashboard requires 
to generate the knowledge graph visualization using vis.js.
"""
def generate_visjs_KG_json(nodes_json_file_path,nodes_json_loaded_file_path):
    
    #Load data from node.json
    with open(nodes_json_file_path, 'r') as myfile:
        data=myfile.read()
        nodes_json_loaded = json.loads(data)
    myfile.close()
    #Cosmetic variables Nodes
    scaling_factor = 2
    min_size = 6
    #"#009BCE" "#0019CE"
    node_colour_engine = "#0019CE"
    node_colour_url = "#0019CE"
    node_colour_user = "#0019CE"

    #Cosmetic variables Relationships
    rel_colour_engine = "#009BCE"
    rel_colour_url = "#009BCE"
    rel_colour_user = "#009BCE"

    rel_colour_engine_highlighted = "#a3dfff"
    rel_colour_url_highlighted = "#a3dfff"
    rel_colour_user_highlighted = "#a3dfff"

    #Operational variables
    node_unique_id = 1
    node_json_output = {}

    #Clears out node_json_output.json of content
    node_json_output_file = open(nodes_json_loaded_file_path, "w")
    node_json_output_file.write("")
    node_json_output_file.close()

    #Opens file and starts of JSON
    node_json_output_file = open(nodes_json_loaded_file_path, "a")
    node_json_output_file.write("{\n\"nodes\": [\n")
    node_json_output_file.close()

    for node_collection in nodes_json_loaded:
        #Populates engine node JSON
        node_json_output_file = open(nodes_json_loaded_file_path, "a")
        node_json_output['id'] = (node_unique_id)
        node_json_output['shape'] =  "circularImage"
        node_json_output['image'] =  "https://github.com/WernerWildenboer/hostimages/blob/master/Images/gear_cropped_3.png?raw=True"
        node_json_output['label'] =  node_collection['engine']
        node_json_output['size'] =  30
        node_json_output['title'] =  node_collection['engine']
        node_json_output['color'] =  node_colour_engine
        json.dump(node_json_output, node_json_output_file, indent=4, sort_keys=True)
        node_json_output_file.write("\n,")

        node_unique_id += 1
        #Populates url node JSON
        node_json_output['id'] = (node_unique_id)
        node_json_output['shape'] =  "circularImage"
        node_json_output['image'] =  "https://github.com/WernerWildenboer/hostimages/blob/master/Images/link_cropped.png?raw=True"
        node_json_output['label'] =  node_collection['url']['url_name']
        node_json_output['size'] =  check_size(min_size,node_collection['url']['total_hits'])*scaling_factor
        node_json_output['title'] =  node_collection['url']['url_name']

        #Checks if a url is anomalous and changes colour to red if true
        if node_collection['url']['anomalous'] == 0:
            node_json_output['color'] =  node_colour_url
        elif node_collection['url']['anomalous'] == 1:
            node_json_output['color'] =  "#F6110D"
        json.dump(node_json_output, node_json_output_file,indent=4, sort_keys=True)
        node_json_output_file.write("\n,")


        node_unique_id += 1
        for unique_user in node_collection['users']:
            #Populates user node JSON
            node_json_output['id'] = (node_unique_id)
            node_json_output['shape'] =  "circularImage"
            node_json_output['image'] =  "https://github.com/WernerWildenboer/hostimages/blob/master/Images/settings_cropped.png?raw=True"
            label_string = "Title : %s \nTotal Hits : %s" % (unique_user['user_name'],int(unique_user['hits']))
            node_json_output['label'] =  label_string 
            node_json_output['size'] =  check_size(min_size,unique_user['hits'])*scaling_factor
            node_json_output['title'] =  unique_user['user_name']
            node_json_output['color'] =  node_colour_user
            node_unique_id += 1
            json.dump(node_json_output, node_json_output_file, indent=4, sort_keys=True)
            #Checks if last element in dict to ensure a extra unnecessary "," is not added
            if node_collection == nodes_json_loaded[-1]:
                node_json_output_file.write("\n")
            else:  
                node_json_output_file.write("\n,")

        node_json_output_file.close()
    node_json_output_file_end = open(nodes_json_loaded_file_path, "a")
    node_json_output_file_end.write("\n],")
    node_json_output_file.close()

    rel_unique_id = 1
    rel_json_output = {}

    node_json_output_file = open(nodes_json_loaded_file_path, "a")
    node_json_output_file.write("\"edges\": [\n")

    for rel_collection in nodes_json_loaded:
         #Populates engine/url relationship JSON
        node_json_output_file = open(nodes_json_loaded_file_path, "a")
        rel_json_output['from'] =rel_unique_id
        rel_unique_id  += 1
        rel_json_output['to'] = rel_unique_id
        rel_json_output['width'] = 2
        rel_json_output['label'] = ""
        enriched_json_color = edict({'color':rel_colour_engine, 'highlight':rel_colour_engine_highlighted})
        rel_json_output['color'] = enriched_json_color
        enriched_json_arrows = edict({'to':{'scaleFactor':'0.5', 'type':"circle"}})
        rel_json_output['arrows'] = enriched_json_arrows
        json.dump(rel_json_output, node_json_output_file, indent=4)
        node_json_output_file.write(",\n")

        user_counter = rel_unique_id 
        for unique_user in rel_collection['users']:
            #Populates user relationship JSON
            user_counter +=1
            rel_json_output['from'] = user_counter
            rel_json_output['to'] = (rel_unique_id)
            rel_json_output['width'] = 2
            rel_json_output['label'] = ""
            enriched_json_color = edict({'color': rel_colour_user , 'highlight':rel_colour_user_highlighted})
            rel_json_output['color'] = enriched_json_color
            enriched_json_arrows = edict({'to':{'scaleFactor':'0.5', 'type':"circle"}})
            rel_json_output['arrows'] = enriched_json_arrows

            json.dump(rel_json_output, node_json_output_file)
            #Checks if last element in dict to ensure a extra unnecessary "," is not added
            if rel_collection == nodes_json_loaded[-1]:
                node_json_output_file.write("\n")
            else:  
                node_json_output_file.write(",\n")


        node_json_output_file.close()
        rel_unique_id  = user_counter +1

    node_json_output_file = open(nodes_json_loaded_file_path, "a")
    #Completes JSON
    node_json_output_file.write("\n]\n}")
    node_json_output_file.close()    


In [174]:
with open('node_json_output.json', 'r') as loaded_file:
    data_reloaded = loaded_file.read()
nodes_json_reloaded = json.loads(data_reloaded)
nodes_json_reloaded

JSONDecodeError: Expecting value: line 818 column 1 (char 24740)

In [9]:
"""
Checks the size of a node and determines 
if it is smaller, larger or equal to the 
minimum value specified and adjusts it 
accordingly.

- node_size > minimum_size : does nothing
- node_size = minimum_size : does nothing 
- node_size < minimum _size : changes node_size to minimum _size
"""
def check_size(min_size,size_input):
    #Convets size_input to int  
    int_size_input = int(size_input)
    #Checks if int_size_input is smaller or 
    #equal to min_size 
    if int_size_input <= min_size:
        return min_size
    else:
        return int_size_input

In [14]:
check_size(11,15)

15

In [13]:
nodes_json_file_path = "nodes.json"
nodes_json_loaded_file_path = "node_json_output.json"
generate_visjs_KG_json(nodes_json_file_path,nodes_json_loaded_file_path)